In [ ]:
import os
import cv2
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout,Flatten
from keras.models import Model, load_model
import tensorflow as tf
from tensorflow.keras import regularizers

In [ ]:
df = pd.read_csv('/kaggle/input/ham1000-segmentation-and-classification/GroundTruth.csv')
classes=df.columns[1:]
classes


In [ ]:
parent_folder="baru"

In [ ]:
if os.path.isdir(parent_folder):
    shutil.rmtree(parent_folder)
os.mkdir(parent_folder)
for i in classes:
    folder=os.path.join(parent_folder,i)
    os.mkdir(folder)

In [ ]:
def preprocessing(image):
    gambar=cv2.imread(f"/kaggle/input/ham1000-segmentation-and-classification/images/{image}.jpg",cv2.IMREAD_COLOR)
    resize=cv2.resize(gambar,[224,224])
    grayScale = cv2.cvtColor(resize, cv2.COLOR_RGB2GRAY )
    #Black hat filter
    kernel = cv2.getStructuringElement(1,(9,9)) 
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    #Gaussian blur
    bhg= cv2.GaussianBlur(blackhat,(3,3),cv2.BORDER_DEFAULT)
    #masking
    ret,mask = cv2.threshold(bhg,10,255,cv2.THRESH_BINARY)
    #Replace pixels of the mask
    dst = cv2.inpaint(resize,mask,6,cv2.INPAINT_TELEA)   
    return dst
for cls in classes:
    images = df[df[cls]==1]['image'].to_list()
    for image in images:
        gambar=preprocessing(image)
        cv2.imwrite(f"baru/{cls}/{image}.jpg",gambar)


In [ ]:

def splitting(location):
    files=[]
    labels=[]
    classes=os.listdir(location)
    for cls in classes:
        path=os.path.join(location,cls)
        list_file=os.listdir(path)
        for file in list_file:
            file_path=os.path.join(path,file)
            files.append(file_path)
            labels.append(cls)
    filepath=pd.Series(files,name="path")
    labelpath=pd.Series(labels,name="label")
    dataset=pd.concat([filepath,labelpath],axis=1)
    strat=dataset['label']
    train_dataset,test_valid_dataset=train_test_split(dataset,train_size=0.8,shuffle=True,random_state=42,stratify=strat)
    strat_test_valid=test_valid_dataset["label"]
    test_dataset,validation_dataset=train_test_split(test_valid_dataset,train_size=0.5,shuffle=True,random_state=42,stratify=strat_test_valid) 
    print('train_df length: ', len(train_dataset), '  test_df length: ',len(test_dataset), '  valid_df length: ', len(validation_dataset))
    print(train_dataset['label'].value_counts())
    return train_dataset,test_dataset,validation_dataset   

In [ ]:
train_dataset,test_dataset,validation_dataset=splitting(parent_folder)

In [ ]:
train_data_count=train_dataset['path'].count()
avg_data_train=int(np.ceil(train_data_count/7))
print(avg_data_train)

In [ ]:
val_data_count=validation_dataset['path'].count()
avg_data_val=int(np.ceil(val_data_count/7))
print(avg_data_val)

In [ ]:
augmented_train_location="augmented_train"
augmented_val_location="augmented_val"
def balancing(dataset,size,train):
    if train==True:
        if os.path.isdir(augmented_train_location):
            shutil.rmtree(augmented_train_location)
        os.mkdir(augmented_train_location)
        for i in dataset['label'].unique():
            os.mkdir(os.path.join(augmented_train_location,i))
    elif train==False:
        if os.path.isdir(augmented_val_location):
            shutil.rmtree(augmented_val_location)
        os.mkdir(augmented_val_location)
        for i in dataset['label'].unique():
            os.mkdir(os.path.join(augmented_val_location,i))
    data=dataset.copy()
    reduce=[]
    grouping=data.groupby("label")
    for i in data['label'].unique():
        group_per_type=grouping.get_group(i)
        if train==True:
            count=len(group_per_type)
            if count > size:
                sampling=group_per_type.sample(size, replace=False, weights=None, random_state=42, axis=0).reset_index(drop=True)
                reduce.append(sampling)
            elif count <=size:
                reduce.append(group_per_type)
        elif train ==False:
            reduce.append(group_per_type)
    reduced_dataset=pd.concat(reduce,axis=0).reset_index(drop=True)
    generator=ImageDataGenerator(horizontal_flip=True,rotation_range=90,width_shift_range=2,height_shift_range=2)
    grouping=reduced_dataset.groupby("label")
    for type in reduced_dataset['label'].unique():
        group_per_type=grouping.get_group(type)
        count=len(group_per_type)
        if count < size:
            augmented_count=0
            delta=size-count
            if train==True:
                location=os.path.join(augmented_train_location,type)
            elif train==False:
                location=os.path.join(augmented_val_location,type)
            augmentation_generator=generator.flow_from_dataframe(group_per_type,x_col='path',y_col=None,target_size=(224,224), class_mode=None,
                                                                  batch_size=1, shuffle=False,save_to_dir=location, save_prefix='augmented_',
                                                                  color_mode='rgb',save_format='jpg')
            while augmented_count<delta:
                images=next(augmentation_generator)
                augmented_count+=len(images)
    files=[]
    labels=[]
    if train==True:
        classes=os.listdir(augmented_train_location)
    elif train==False:
        classes=os.listdir(augmented_val_location)
    for cls in classes:
        if train==True:
            path=os.path.join(augmented_train_location,cls)
        elif train==False:
            path=os.path.join(augmented_val_location,cls)
        list_file=os.listdir(path)
        for file in list_file:
            file_path=os.path.join(path,file)
            files.append(file_path)
            labels.append(cls)
    filepath=pd.Series(files,name="path")
    labelpath=pd.Series(labels,name="label")
    augmented_dataset=pd.concat([filepath,labelpath],axis=1)
    combined_dataset=pd.concat([reduced_dataset,augmented_dataset],axis=0).reset_index(drop=True)
    return combined_dataset


In [ ]:
# balanced_train_dataset=balancing(train_dataset,avg_data_train,True)
# balanced_train_dataset=balancing(train_dataset,800)
# print(balanced_train_dataset['label'].value_counts())
balanced_train_dataset=train_dataset
# balanced_val=balancing(validation_dataset,avg_data_val,False)

In [ ]:
batch_size=32
img_size=(224,224)
img_shape=(224,224,3)
length=len(test_dataset)
test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=64],reverse=True)[0]  
test_steps=int(length/test_batch_size)
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps)
def process(image):
    return image
generator_for_training=ImageDataGenerator(preprocessing_function=process,horizontal_flip=True)
generator=ImageDataGenerator(preprocessing_function=process)

train_generator=generator_for_training.flow_from_dataframe(balanced_train_dataset,x_col='path',y_col='label',
                                                           target_size=img_size,class_mode='categorical',color_mode='rgb',
                                                           shuffle=True,batch_size=batch_size)
validation_generator=generator.flow_from_dataframe(validation_dataset,x_col='path',y_col='label',
                                                   target_size=img_size,class_mode='categorical',color_mode='rgb',
                                                   shuffle=True,batch_size=batch_size)
test_generator=generator.flow_from_dataframe(test_dataset,x_col='path',y_col='label', 
                                             target_size=img_size,class_mode='categorical',color_mode='rgb',
                                             shuffle=False,batch_size=test_batch_size)
train_steps=int(np.ceil(len(train_generator.labels)/batch_size))

In [ ]:
learning_rate=1e-4
epoch=30
factor=learning_rate/epoch
def efficientnet_b4():
    input=Input(shape=img_shape)
    base=tf.keras.applications.EfficientNetB4(weights='imagenet',include_top=False,input_shape=img_shape,input_tensor=input,classes=7)
#     base.trainable=False
#     x=Flatten()(base.output)
    x = GlobalAveragePooling2D()(base.output)
    x = BatchNormalization()(x)
#     x = Dropout(0.2)(x)
    x = Dropout(0.5)(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.5)(x)
    output = Dense(7, activation='softmax',kernel_regularizer=regularizers.L1L2(l1=0.01, l2=0.01))(x)
#     output = Dense(7, activation='softmax')(x)
    model = Model(input, output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=['accuracy'])
    return model
efficientnet_b4_model=efficientnet_b4()

In [ ]:
# efficientnet_b4_model.summary()

In [ ]:
#stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=True,verbose=1,patience=5) 
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=factor,patience=5, min_lr=learning_rate,verbose=1) 
checkpoint=tf.keras.callbacks.ModelCheckpoint("efficientnet_b4_model.h5",monitor="val_loss",mode="min",save_best_only = True,verbose=1)
callback_list = [ reduce_lr, checkpoint]

In [ ]:
classifier_history = efficientnet_b4_model.fit(train_generator, batch_size=batch_size,
                validation_data=validation_generator,
                steps_per_epoch=train_steps,
                validation_steps=None,
                epochs=epoch,
                callbacks=callback_list
)

In [ ]:
def plot_history(history, title, plot_type, xlabel, ylabel):
    plt.figure(figsize=(12,8))
    if plot_type == 'accuracy':
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.legend(['Train Accuracy', 'Validation Accuracy'], loc='upper left')
    else:
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.legend(['Train Loss', 'Validation Loss'], loc='upper left')
    plt.title('Model Accuracy')
    plt.ylabel('{}'.format(ylabel))
    plt.xlabel('{}'.format(xlabel))
    plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
    plt.title('{}'.format(title))
    plt.savefig('{}.png'.format(title))
    plt.show()

In [ ]:
plot_history(classifier_history, 'Accuracy', 'accuracy', 'Epochs', 'Model Accuracy')

In [ ]:
plot_history(classifier_history, 'Loss', 'loss', 'Epochs', 'Loss')

In [ ]:
acc=efficientnet_b4_model.evaluate( test_generator, batch_size=test_batch_size, verbose=1, steps=test_steps, return_dict=False)[1]*100
print(f'accuracy on the test set is {acc:5.2f} %')

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
def print_info( test_gen, preds, print_code, save_dir, subject ):
    class_dict=test_gen.class_indices
    labels= test_gen.labels
    file_names= test_gen.filenames 
    error_list=[]
    true_class=[]
    pred_class=[]
    prob_list=[]
    new_dict={}
    error_indices=[]
    y_pred=[]
    for key,value in class_dict.items():
        new_dict[value]=key             # dictionary {integer of class number: string of class name}
    # store new_dict as a text fine in the save_dir
    classes=list(new_dict.values())     # list of string of class names     
    errors=0      
    for i, p in enumerate(preds):
        pred_index=np.argmax(p)         
        true_index=labels[i]  # labels are integer values
        if pred_index != true_index: # a misclassification has occurred
            error_list.append(file_names[i])
            true_class.append(new_dict[true_index])
            pred_class.append(new_dict[pred_index])
            prob_list.append(p[pred_index])
            error_indices.append(true_index)            
            errors=errors + 1
        y_pred.append(pred_index)    
    if print_code !=0:
        if errors>0:
            if print_code>errors:
                r=errors
            else:
                r=print_code           
            msg='{0:^28s}{1:^28s}{2:^28s}{3:^16s}'.format('Filename', 'Predicted Class' , 'True Class', 'Probability')
            print_in_color(msg, (0,255,0),(55,65,80))
            for i in range(r):                
                split1=os.path.split(error_list[i])                
                split2=os.path.split(split1[0])                
                fname=split2[1] + '/' + split1[1]
                msg='{0:^28s}{1:^28s}{2:^28s}{3:4s}{4:^6.4f}'.format(fname, pred_class[i],true_class[i], ' ', prob_list[i])
                print_in_color(msg, (255,255,255), (55,65,60))
                #print(error_list[i]  , pred_class[i], true_class[i], prob_list[i])               
        else:
            msg='With accuracy of 100 % there are no errors to print'
            print_in_color(msg, (0,255,0),(55,65,80))
    if errors>0:
        plot_bar=[]
        plot_class=[]
        for  key, value in new_dict.items():        
            count=error_indices.count(key) 
            if count!=0:
                plot_bar.append(count) # list containg how many times a class c had an error
                plot_class.append(value)   # stores the class 
        fig=plt.figure()
        fig.set_figheight(len(plot_class)/3)
        fig.set_figwidth(10)
        plt.style.use('fivethirtyeight')
        for i in range(0, len(plot_class)):
            c=plot_class[i]
            x=plot_bar[i]
            plt.barh(c, x, )
            plt.title( ' Errors by Class on Test Set')
    y_true= np.array(labels)        
    y_pred=np.array(y_pred)
    if len(classes)<= 30:
        # create a confusion matrix 
        cm = confusion_matrix(y_true, y_pred )        
        length=len(classes)
        if length<8:
            fig_width=8
            fig_height=8
        else:
            fig_width= int(length * .5)
            fig_height= int(length * .5)
        plt.figure(figsize=(fig_width, fig_height))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
        plt.xticks(np.arange(length)+.5, classes, rotation= 90)
        plt.yticks(np.arange(length)+.5, classes, rotation=0)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
    clr = classification_report(y_true, y_pred, target_names=classes)
    print("Classification Report:\n----------------------\n", clr)

In [ ]:
print_code=0
subject='skin disease'
preds=efficientnet_b4_model.predict(test_generator, steps=test_steps, verbose=1) 
print_info( test_generator, preds, print_code, r'./', subject )  

In [ ]:
efficientnet_b4_model.save("efficientnet_b4_model_coba.h5")